# Synthetically Generated Data

We test the potential of our estimation algorithm by synthetically generated data.

In [1]:
# std library
import os
import json
from pathlib import Path
from pprint import pprint
from datetime import datetime, timedelta
from typing import Sequence

# third-party
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from numpy import typing as npt

# local
from synthetic_data import synthetic_data_simulation  # type: ignore

roundint = lambda x: int(round(x))

### 1. Model Setting

In [2]:
# Capacities & Total Innovation Uncertainty
param_theta = 1.0

param_c_i = 1.2
param_c_j = 1.5
param_sigma = 2.0
param_lambda = 1.0
param_r = 15.0

# Contest Time Duaration
start_time = datetime(2025, 1, 1, 0, 0, 0)
end_time = datetime(2025, 4, 1, 0, 0, 0)
contest_days = (end_time - start_time).days
contest_seconds = int((end_time - start_time).total_seconds())
contest_hours = int(contest_seconds / 3600)
contest_timegrids: list[datetime] = np.arange(start_time, end_time, \
		timedelta(hours=1), dtype=datetime).tolist()
time_unit_2f = 1 / 24

print(contest_days, contest_hours, len(contest_timegrids))

90 2160 2160


### 2. Simulations

In [3]:
number_of_samples = 20

In [4]:
combined_data = {
	'N_tests': number_of_samples,
	'theta': param_theta,
	'ratio': param_r,
	'Delta2f': time_unit_2f,
	'N_Delta': roundint((end_time - start_time).total_seconds() / 3600),
	'Ni_arr': [],
	'Nj_arr': [],
	'Ni_max': 0,
	'Nj_max': 0,
	'hat_t_i_arr': [],
	'hat_t_j_arr': [],
	'hat_y_arr': [],
	#'efforts_i_arr': [],
	#'efforts_j_arr': [],
}

In [5]:
Ni_max = 0
Nj_max = 0

for ii in range(number_of_samples):
	random_seed = 1000 + 3 * ii
	time_grids, i_effort_dynamic, j_effort_dynamic, \
	real_gap_dynamic, perceived_gap_dynamic, observed_gap_dynamic, \
	observed_i_commits, observed_j_commits = \
		synthetic_data_simulation(
			theta=param_theta,
			c_i=param_c_i,
			c_j=param_c_j,
			sigma=param_sigma,
			lamb=param_lambda,
			intensity_effort_ratio=param_r,
			hour_arrival_ub=1.0,
			start_time=start_time,
			end_time=end_time,
			time_unit=timedelta(hours=1),
			time_unit_2f=time_unit_2f,
			approx = True,  # using approximated version by default
			seed_brownian = random_seed + 0,
			seed_poisson = random_seed + 1,
			seed_uniform = random_seed + 2,
		)
	# pprint(observed_gap_dynamic)
	Ni = len(observed_i_commits)
	Nj = len(observed_j_commits)
	if Ni > Ni_max:
		Ni_max = Ni
	if Nj > Nj_max:
		Nj_max = Nj
	hat_t_i = [(dt - start_time).total_seconds() / 3600 for dt in observed_i_commits]
	hat_t_j = [(dt - start_time).total_seconds() / 3600 for dt in observed_j_commits]
	hat_y = observed_gap_dynamic.tolist()
	efforts_i = i_effort_dynamic.tolist()
	efforts_j = j_effort_dynamic.tolist()
	combined_data['Ni_arr'].append(Ni)
	combined_data['Nj_arr'].append(Nj)
	combined_data['hat_t_i_arr'].append(hat_t_i)
	combined_data['hat_t_j_arr'].append(hat_t_j)
	combined_data['hat_y_arr'].append(hat_y)
combined_data['Ni_max'] = Ni_max
combined_data['Nj_max'] = Nj_max

In [6]:
for idx_test in range(number_of_samples):
	# hat_t_i_arr
	hat_t_i = combined_data['hat_t_i_arr'][idx_test]
	hat_t_i_len = len(hat_t_i)
	if hat_t_i_len < Ni_max:
		hat_t_i.extend([0] * (Ni_max - hat_t_i_len))
	# hat_t_j_arr
	hat_t_j = combined_data['hat_t_j_arr'][idx_test]
	hat_t_j_len = len(hat_t_j)
	if hat_t_j_len < Nj_max:
		hat_t_j.extend([0] * (Nj_max - hat_t_j_len))

In [7]:
# save the observed data
wd = os.getcwd()
wd_synthetic_data = os.path.join(wd, f'data_{contest_days}_combined_{number_of_samples}.json')

with open(wd_synthetic_data, 'w') as f:
	json.dump(combined_data, f, indent=4)

### 3. Bayesian Inference

In [8]:
# import cmdstanpy
# cmdstanpy.install_cmdstan()
from cmdstanpy import CmdStanModel, compile_stan_file

In [9]:
# build stan model
stan_file = os.path.join(wd, f'synthetic_data_90_replicated.stan')
output_dir = Path('./tmp')
model = CmdStanModel(stan_file=stan_file)

In [10]:
# fit the model with data
fit = model.sample( \
		data=wd_synthetic_data,
		iter_warmup=1000,
		iter_sampling=2000,
		chains=4,
		parallel_chains=4,
		show_console=False,
		#max_treedepth=12,  # for difficult model
		#adapt_delta=0.99,  # for difficult model
		output_dir=output_dir,
		seed=12345,
	)

16:10:39 - cmdstanpy - INFO - CmdStan start processing


chain 1 |          | 00:00 Status

chain 2 |          | 00:00 Status

chain 3 |          | 00:00 Status

chain 4 |          | 00:00 Status

16:29:43 - cmdstanpy - INFO - CmdStan done processing.


In [11]:
# pprint(fit.diagnose())

In [12]:
posteriors = fit.stan_variables()

In [13]:
def rmse(true, mean, std):
	return ((true - mean)**2 + std**2)**0.5

def display_estimation_results(var_names: list[str], true_vals, posteriors):
	data = []
	for name, true_val in zip(var_names, true_vals):
		posterior_mean = posteriors[name].mean()
		posterior_std = posteriors[name].std()
		record = (name, true_val, posterior_mean, posterior_std, rmse(true_val, posterior_mean, posterior_std))
		data.append(record)
	columns = ['Name', 'True Val.', 'Posterior Mean', 'Posterior Std.', 'RMSE']
	return pd.DataFrame(data, columns=columns)

In [14]:
df_posteriors = display_estimation_results(
	['c_i', 'c_j', 'sigma', 'lambda'],
	[param_c_i, param_c_j, param_sigma, param_lambda],
	posteriors
)
df_posteriors.to_csv(f'contest_90_posteriors_{number_of_samples}_tests.csv', index=False)
df_posteriors

,Name,True Val.,Posterior Mean,Posterior Std.,RMSE
0,c_i,1.2,1.205638,0.059447,0.059714
1,c_j,1.5,1.638771,0.089548,0.165155
2,sigma,2.0,1.980652,0.068660,0.071334
3,lambda,1.0,0.989711,0.059499,0.060382


Compare posterior `m[1]..m[T]` with real

### Clean Up

In [15]:
for file in output_dir.iterdir():
	if file.is_file():
		file.unlink()